# Space Images Classifier - Using Kaggle dataset

https://www.kaggle.com/datasets/abhikalpsrivastava15/space-images-category?utm_source=chatgpt.com

### Notebook 5 - Training stage 2 (Unfrozen backbone for fine-tuning)

# Import librairies

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os
import json

# Add the root folder to Python's module search path
sys.path.append(os.path.abspath(os.path.join(".."))) 
# Import the project configuration
from config import DEVICE, OUTPUT_PATH, BATCH_SIZE, NUM_WORKERS, EPOCHS_STAGE2, LEARNING_RATE_STAGE2
from models import SpaceClassifier
from train_utils import train_epoch, validate
from datasets import SpaceImageDataset, train_transforms, val_test_transforms

import shutil
from pathlib import Path
import cv2
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision.transforms.functional as TF

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.utils import class_weight

# Load callback/config variables

In [2]:
ROOT_PATH = Path("..")
# Path to load JSON
CONFIG_JSON_PATH = ROOT_PATH / "config_dynamic.json"

try:
    with open(CONFIG_JSON_PATH) as f:
        dynamic_config = json.load(f)
except FileNotFoundError:
    dynamic_config = {}
    
NUM_CLASSES = dynamic_config.get("NUM_CLASSES", 0)
class_names = dynamic_config.get("class_names", [])
split_success = dynamic_config.get("split_success", 5)

# Path to the saved tensor
WEIGHTS_PATH = Path("models") / "class_weights_tensor.pth"

# Load the tensor
class_weights_tensor = torch.load(WEIGHTS_PATH, map_location=DEVICE)



# Training stage 2 with unfrozen backbone

## Load the best model

In [3]:
# Create model
model = SpaceClassifier(NUM_CLASSES, pretrained=True)
model = model.to(DEVICE)

print("=" * 80)
print(f"Model created: EfficientNet-B0")
print(f"Device: {DEVICE}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print("-" * 80)

# Load best model
model.load_state_dict(torch.load('models/stage1_best.pth'))
print("Best model from stage 1 loaded")
# Load history of the stage1
history_stage1 = torch.load('models/history_stage1_del_img.pt')
print("History of the stage 1 loaded")
print("=" * 80)

Model created: EfficientNet-B0
Device: mps
Total parameters: 4,796,290
Trainable parameters: 788,742
--------------------------------------------------------------------------------
Best model from stage 1 loaded
History of the stage 1 loaded


## Create Dataset

In [4]:
train_dataset = SpaceImageDataset(OUTPUT_PATH / "train", transform=train_transforms)
val_dataset = SpaceImageDataset(OUTPUT_PATH / "validation", transform=val_test_transforms)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Dataset created from ../space_images_split/train
   -> 765 valid images
7 images skipped (see ../space_images_split/train/skipped_images.txt)
Dataset created from ../space_images_split/validation
   -> 161 valid images
2 images skipped (see ../space_images_split/validation/skipped_images.txt)


## Create Data loaders

In [5]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True, 
    num_workers=NUM_WORKERS,
    pin_memory=True if DEVICE.type == 'mps' else False
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=NUM_WORKERS,
    pin_memory=True if DEVICE.type == 'mps' else False
)

## Training the model - Phase 2

In [ ]:
if split_success and history_stage1:
    # Loss and optimizer
    # Add label_smoothing argument to help generalization and better handle noisy labels
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=0.1)

    # # Unfreeze backbone
    # model.unfreeze_backbone()
    # print(f"Backbone unfrozen")

    # # Unfreeze partial backbone (only the last block)
    # model.unfreeze_backbone_partial()

    # Unfreeze for stage 2
    model.unfreeze_backbone_stage(stage=2)

    # # Add weight_decay in order to add L2 regularization > it penalizes large weights
    # optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE_STAGE2, weight_decay=1e-3)

    # === Differential Learning Rates ===
    backbone_params = [p for n, p in model.backbone.named_parameters() if "features" in n and p.requires_grad]
    classifier_params = [p for n, p in model.backbone.named_parameters() if "classifier" in n and p.requires_grad]

    optimizer = torch.optim.Adam([
        {'params': backbone_params, 'lr': 1e-6, 'weight_decay': 1e-3},
        {'params': classifier_params, 'lr': 1e-4, 'weight_decay': 1e-3}
    ])

    # Automatically reduces the learning rate by 50% if validation loss stops improving for 5 epochs
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Trainable parameters: {trainable_params:,}")
    
    print("=" * 80)
    print(f"Starting Stage 2 Training")
    print("-" * 80)
    print(f"Estimated time on M1: 20-40 minutes\n")
    print("=" * 80)   

    history_stage2 = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    best_val_acc = 0.0
    patience_counter = 0
    PATIENCE = 6
    
    for epoch in range(EPOCHS_STAGE2):
        print("=" * 80)
        print(f"Epoch {epoch+1}/{EPOCHS_STAGE2}")
        print("-" * 60)
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
        val_loss, val_acc, _, _ = validate(model, val_loader, criterion, DEVICE)
        
        scheduler.step(val_loss)
        
        history_stage2['train_loss'].append(train_loss)
        history_stage2['train_acc'].append(train_acc)
        history_stage2['val_loss'].append(val_loss)
        history_stage2['val_acc'].append(val_acc)
        
        print(f"\nTrain Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%")
        print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc*100:.2f}%")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'models/final_best.pth')
            print(f"Best model saved! (Val Acc: {val_acc*100:.2f}%)")
            # Save stage 2 logs to the disk
            torch.save(history_stage2, 'models/history_stage2.pt')
            patience_counter = 0
        else:
            patience_counter += 1
        
        if patience_counter >= PATIENCE:
            print(f"\nEarly stopping triggered (patience={PATIENCE})")
            break
    
    print("\nStage 2 fine-tuning complete!")
    
    # Load best model
    model.load_state_dict(torch.load('models/final_best.pth'))

else:
    history_stage2 = None

print("=" * 80)

TypeError: SpaceClassifier.unfreeze_backbone_stage() got multiple values for argument 'stage'

# End of notebook 5 - Training stage 2